In [1]:
import pandas as pd
import datetime as dt

In [2]:
data = pd.read_excel('WCSURGERY.xlsx')
Revenue_Codes = pd.read_excel('Revenue Codes.xlsx')
RUBIN = pd.read_excel('WCRUBINPRECERTSURG.xlsx')

In [3]:
filtered_data = data[data['Claim Number'].isin(RUBIN['Claim Number'])]
data = filtered_data
data

,Transaction Date,Service Start,Service End,Claim Status,Claim Number,Coverage Desc,Date of Incident/Loss,Financial Category,Payee,Transaction Type,...,In or Out Patient Desc,Paid Amount,Revenue Code,Provider Practice Address 1,Procedure Code 1,Procedure Description 1,Procedure Code 2,Procedure Description 2,Procedure Type,EDI Body Part Desc
4,2022-01-03,2021-11-01 00:00:00,2021-11-22 00:00:00,FINAL,WDWW2021219859,Workers Comp/Crew - Lost Time Treatment,2021-08-06,Med/BI/Comp Paid,SELECT PHYSICAL THERAPY HOLDINGS IN,1P12,...,NaN,0.0,NaN,12500 STATE ROAD 535,97014,ELECTRIC STIMULATION THERAPY,NaN,NaN,NaN,Multiple Body Parts
5,2022-01-03,2021-11-01 00:00:00,2021-11-22 00:00:00,FINAL,WDWW2021219859,Workers Comp/Crew - Lost Time Treatment,2021-08-06,Med/BI/Comp Paid,SELECT PHYSICAL THERAPY HOLDINGS IN,1P12,...,NaN,0.0,NaN,12500 STATE ROAD 535,97035,ULTRASOUND THERAPY,NaN,NaN,NaN,Multiple Body Parts
6,2022-01-03,2021-11-01 00:00:00,2021-11-22 00:00:00,FINAL,WDWW2021219859,Workers Comp/Crew - Lost Time Treatment,2021-08-06,Med/BI/Comp Paid,SELECT PHYSICAL THERAPY HOLDINGS IN,1P12,...,NaN,360.0,NaN,12500 STATE ROAD 535,97110,THERAPEUTIC EXERCISES,NaN,NaN,NaN,Multiple Body Parts
7,2022-01-03,2021-11-01 00:00:00,2021-11-22 00:00:00,FINAL,WDWW2021219859,Workers Comp/Crew - Lost Time Treatment,2021-08-06,Med/BI/Comp Paid,SELECT PHYSICAL THERAPY HOLDINGS IN,1P12,...,NaN,0.0,NaN,12500 STATE ROAD 535,97140,MANUAL THERAPY,NaN,NaN,NaN,Multiple Body Parts
8,2022-01-03,2021-11-01 00:00:00,2021-11-22 00:00:00,FINAL,WDWW2021219859,Workers Comp/Crew - Lost Time Treatment,2021-08-06,Med/BI/Comp Paid,SELECT PHYSICAL THERAPY HOLDINGS IN,1P12,...,NaN,0.0,NaN,12500 STATE ROAD 535,97530,THERAPEUTIC ACTIVITIES,NaN,NaN,NaN,Multiple Body Parts
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182772,2024-11-25,2024-11-15 00:00:00,2024-11-28 00:00:00,OPEN,WDWW2024236673,Workers Comp/Crew - Lost Time Treatment,2024-01-29,IND/PD/Coll Paid,"Beaufils, Irma J",3P09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Finger(s)
182799,2024-11-25,2024-11-19 00:00:00,2024-12-02 00:00:00,OPEN,WDWW2024237365,Workers Comp/Crew - Lost Time Treatment,2024-01-14,IND/PD/Coll Paid,"Chester, Sarah Y",3P09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Finger(s)
182935,2024-11-26,2024-11-18 00:00:00,2024-11-18 00:00:00,OPEN,WDWW2024237365,Workers Comp/Crew - Lost Time Treatment,2024-01-14,Med/BI/Comp Paid,JERRY A RUBIN MD PA,1P41,...,NaN,90.0,NaN,6900 TURKEY LAKE RD STE 1-7,99455,DISABILITY EXAMINATION,NaN,NaN,NaN,Finger(s)
183277,2024-12-03,2024-10-18 00:00:00,2024-11-28 00:00:00,OPEN,WDWW2024236673,Workers Comp/Crew - Lost Time Treatment,2024-01-29,IND/PD/Coll Paid,"Beaufils, Irma J",3P09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Finger(s)


In [4]:
def calculate_fiscal_year(transaction_date):
    fiscal_year_start_month = 10  # Fiscal year starts in October
    if transaction_date.month >= fiscal_year_start_month:
        return transaction_date.year + 1
    else:
        return transaction_date.year
    
data['Fiscal Year'] = data['Transaction Date'].apply(calculate_fiscal_year)

In [5]:
data['Procedure Code 1'] = pd.to_numeric(data['Procedure Code 1'], errors='coerce')

In [6]:
# Step 1: Identify Surgery CPT Codes
data['Procedure Code 1'] = pd.to_numeric(data['Procedure Code 1'], errors='coerce')
surgery_data = data[(data['Procedure Code 1'] >= 10004) & (data['Procedure Code 1'] <= 69990)].copy()

# Step 2: Sort Surgeries by Date and Assign Order
surgery_data = surgery_data.sort_values(by=['Claim Number', 'Service Start'])
surgery_data['Surgery Order'] = surgery_data.groupby('Claim Number').cumcount() + 1

# Step 3: Calculate Total Cost for Each Surgery
results = []

for _, surgery_row in surgery_data.iterrows():
    claim_number = surgery_row['Claim Number']
    surgery_date = surgery_row['Service Start']
    surgery_cpt = surgery_row['Procedure Code 1']
    surgery_order = surgery_row['Surgery Order']
    
    # Filter rows for the same claim within 1 day before/after the surgery date
    relevant_transactions = data[
        (data['Claim Number'] == claim_number) &
        (data['Service Start'] >= surgery_date - pd.Timedelta(days=1)) &
        (data['Service Start'] <= surgery_date + pd.Timedelta(days=1))
    ]
    
    # Sum the Paid Amount
    total_cost = relevant_transactions['Paid Amount'].sum()
    
    # Append results
    results.append({
        'Claim Number': claim_number,
        'Surgery CPT Code': surgery_cpt,
        'Surgery Order': f'Surgery {surgery_order}',
        'Total Cost (1 Day Before/After)': total_cost
    })

# Convert results to a DataFrame
final_results = pd.DataFrame(results)

In [7]:
final_results

,Claim Number,Surgery CPT Code,Surgery Order,Total Cost (1 Day Before/After)
0,WDWW2021217954,26055.0,Surgery 1,2379.86
1,WDWW2021217954,26055.0,Surgery 2,2379.86
2,WDWW2021218441,25000.0,Surgery 1,2651.56
3,WDWW2021218441,25000.0,Surgery 2,2651.56
4,WDWW2021218501,62321.0,Surgery 1,1475.81
...,...,...,...,...
711,WDWW2024238793,35207.0,Surgery 5,12197.92
712,WDWW2024238793,64417.0,Surgery 6,12197.92
713,WDWW2024238793,26356.0,Surgery 7,12197.92
714,WDWW2024238793,64831.0,Surgery 8,12197.92


In [9]:
specific_claim_number = 'WDWW2021217954'  # Example claim number

# Filter rows with the specific claim number
claim_data = data[data['Claim Number'] == specific_claim_number]

In [10]:
claim_data

,Transaction Date,Service Start,Service End,Claim Status,Claim Number,Coverage Desc,Date of Incident/Loss,Financial Category,Payee,Transaction Type,...,Paid Amount,Revenue Code,Provider Practice Address 1,Procedure Code 1,Procedure Description 1,Procedure Code 2,Procedure Description 2,Procedure Type,EDI Body Part Desc,Fiscal Year
3400,2022-01-28,2022-01-18 00:00:00,2022-01-18 00:00:00,FINAL,WDWW2021217954,Workers Comp/Crew - Lost Time Treatment,2021-03-06,Med/BI/Comp Paid,JERRY A RUBIN MD PA,1P41,...,10.00,NaN,6900 TURKEY LAKE RD1 7,73130.0,X-RAY EXAM OF HAND,NaN,NaN,NaN,Multiple Body Parts,2022
3401,2022-01-28,2022-01-18 00:00:00,2022-01-18 00:00:00,FINAL,WDWW2021217954,Workers Comp/Crew - Lost Time Treatment,2021-03-06,Med/BI/Comp Paid,JERRY A RUBIN MD PA,1P41,...,182.00,NaN,6900 TURKEY LAKE RD1 7,99204.0,OFFICE O/P NEW MOD 45-59 MIN,NaN,NaN,NaN,Multiple Body Parts,2022
6276,2022-02-21,2022-02-09 00:00:00,2022-02-09 00:00:00,FINAL,WDWW2021217954,Workers Comp/Crew - Lost Time Treatment,2021-03-06,Med/BI/Comp Paid,JERRY A RUBIN MD PA,1P41,...,118.00,NaN,6900 TURKEY LAKE RD1 7,99214.0,OFFICE O/P EST MOD 30-39 MIN,NaN,NaN,NaN,Multiple Body Parts,2022
9324,2022-03-16,2022-03-08 00:00:00,2022-03-08 00:00:00,FINAL,WDWW2021217954,Workers Comp/Crew - Lost Time Treatment,2021-03-06,Med/BI/Comp Paid,JERRY A RUBIN MD PA,1P41,...,80.00,NaN,6900 TURKEY LAKE RD1 7,99213.0,OFFICE O/P EST LOW 20-29 MIN,NaN,NaN,NaN,Multiple Body Parts,2022
9609,2022-03-18,2022-03-10 00:00:00,2022-03-23 00:00:00,FINAL,WDWW2021217954,Workers Comp/Crew - Lost Time Treatment,2021-03-06,IND/PD/Coll Paid,"Galvan, Diana G",3P06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Multiple Body Parts,2022
10315,2022-03-25,2022-03-10 00:00:00,2022-03-10 00:00:00,FINAL,WDWW2021217954,Workers Comp/Crew - Lost Time Treatment,2021-03-06,Med/BI/Comp Paid,JERRY A RUBIN MD PA,1P41,...,443.00,NaN,6900 TURKEY LAKE RD2 5,26055.0,INCISE FINGER TENDON SHEATH,NaN,NaN,NaN,Multiple Body Parts,2022
13078,2022-04-14,2022-03-24 00:00:00,2022-04-02 00:00:00,FINAL,WDWW2021217954,Workers Comp/Crew - Lost Time Treatment,2021-03-06,IND/PD/Coll Paid,"Galvan, Diana G",3P07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Multiple Body Parts,2022
13079,2022-04-14,2022-03-24 00:00:00,2022-04-02 00:00:00,FINAL,WDWW2021217954,Workers Comp/Crew - Lost Time Treatment,2021-03-06,IND/PD/Coll Paid,"Galvan, Diana G",3P22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Multiple Body Parts,2022
13825,2022-04-21,2022-04-03 00:00:00,2022-04-16 00:00:00,FINAL,WDWW2021217954,Workers Comp/Crew - Lost Time Treatment,2021-03-06,IND/PD/Coll Paid,"Galvan, Diana G",3P07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Multiple Body Parts,2022
16340,2022-05-05,2022-04-17 00:00:00,2022-04-18 00:00:00,FINAL,WDWW2021217954,Workers Comp/Crew - Lost Time Treatment,2021-03-06,IND/PD/Coll Paid,"Galvan, Diana G",3P07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Multiple Body Parts,2022
